In [4]:
import torch



In [5]:
inputs=torch.tensor(
    [
     [0.43,0.15,0.89],#your
     [0.55,0.87,0.66],#journey
     [0.57,0.85,0.64],#starts
     [0.22,0.58,0.33],#with
     [0.77,0.25,0.10],#one
     [0.05,0.80,0.55],#step
    ]
)

In [6]:
w_q=torch.nn.Linear(inputs.shape[1],2,bias=False)
print(w_q)
w_k=torch.nn.Linear(inputs.shape[1],2,bias=False)
w_v=torch.nn.Linear(inputs.shape[1],2,bias=False)

Linear(in_features=3, out_features=2, bias=False)


In [7]:
queries=w_q(inputs)
keys=w_k(inputs)
values=w_v(inputs)

print(queries)

tensor([[-0.3270, -0.5156],
        [-0.0844, -0.7772],
        [-0.0986, -0.7681],
        [ 0.0397, -0.4323],
        [-0.3273, -0.3878],
        [ 0.1701, -0.5485]], grad_fn=<MmBackward0>)


In [8]:
attn_score=queries@keys.T
print(attn_score)

tensor([[ 1.7913e-01,  1.3354e-01,  1.3731e-01,  4.4606e-02,  1.6665e-01,
          1.4401e-02],
        [ 3.1231e-01,  7.8952e-02,  7.8960e-02, -2.0569e-04,  5.7149e-02,
          5.0050e-03],
        [ 3.0707e-01,  8.2581e-02,  8.2800e-02,  2.3079e-03,  6.3705e-02,
          5.5682e-03],
        [ 1.8268e-01,  1.7962e-02,  1.6763e-02, -1.4421e-02, -9.3755e-03,
         -7.5910e-04],
        [ 1.2628e-01,  1.2482e-01,  1.2879e-01,  4.6993e-02,  1.6402e-01,
          1.4160e-02],
        [ 2.4420e-01, -1.3052e-02, -1.6234e-02, -3.8059e-02, -6.8750e-02,
         -5.8567e-03]], grad_fn=<MmBackward0>)


In [9]:
attn_weight=torch.softmax(attn_score/keys.shape[1]**0.5,dim=1)
print(attn_weight)

tensor([[0.1745, 0.1690, 0.1694, 0.1587, 0.1730, 0.1553],
        [0.1947, 0.1651, 0.1651, 0.1561, 0.1625, 0.1566],
        [0.1937, 0.1653, 0.1653, 0.1561, 0.1631, 0.1565],
        [0.1852, 0.1648, 0.1647, 0.1611, 0.1617, 0.1626],
        [0.1696, 0.1694, 0.1699, 0.1603, 0.1742, 0.1567],
        [0.1952, 0.1627, 0.1623, 0.1599, 0.1564, 0.1635]],
       grad_fn=<SoftmaxBackward0>)


# Create Mask where the value above the diagonal is zero


In [10]:
#use pytorch tril function to diagonalize
context_length=inputs.shape[0]
mask1=torch.tril(torch.ones(context_length,context_length))
print(mask1)

tensor([[1., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1.]])


In [11]:
#use element wise multiplication not matrix
masked_attn_score=attn_weight*mask1
print(masked_attn_score)

#but this attention score is not fully masked or optimal becaus the softmax normalization adds influence on its from the previous attn score so use upper negative infinity mask instead of zero

tensor([[0.1745, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1947, 0.1651, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1937, 0.1653, 0.1653, 0.0000, 0.0000, 0.0000],
        [0.1852, 0.1648, 0.1647, 0.1611, 0.0000, 0.0000],
        [0.1696, 0.1694, 0.1699, 0.1603, 0.1742, 0.0000],
        [0.1952, 0.1627, 0.1623, 0.1599, 0.1564, 0.1635]],
       grad_fn=<MulBackward0>)


In [12]:
#normalize the masked attention score  to sum up 1 in each row to get masked attn weight
row_sum=masked_attn_score.sum(dim=1,keepdim=True)
print(row_sum)
masked_attn_weight=masked_attn_score/row_sum
print(masked_attn_weight)

tensor([[0.1745],
        [0.3597],
        [0.5243],
        [0.6757],
        [0.8433],
        [1.0000]], grad_fn=<SumBackward1>)
tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5412, 0.4588, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3695, 0.3152, 0.3153, 0.0000, 0.0000, 0.0000],
        [0.2740, 0.2439, 0.2437, 0.2384, 0.0000, 0.0000],
        [0.2011, 0.2009, 0.2014, 0.1901, 0.2065, 0.0000],
        [0.1952, 0.1627, 0.1623, 0.1599, 0.1564, 0.1635]],
       grad_fn=<DivBackward0>)


In [13]:
masked_context_vectore=masked_attn_weight@values
print(masked_context_vectore)

tensor([[ 1.1012e-04, -1.6020e-01],
        [-7.1134e-02, -2.2580e-01],
        [-1.0036e-01, -2.4875e-01],
        [-9.5312e-02, -2.3328e-01],
        [-1.3272e-01, -2.1261e-01],
        [-1.0512e-01, -2.1596e-01]], grad_fn=<MmBackward0>)


# Negative infinity masking to avoid any influence of future attention

In [14]:
print(attn_score)

tensor([[ 1.7913e-01,  1.3354e-01,  1.3731e-01,  4.4606e-02,  1.6665e-01,
          1.4401e-02],
        [ 3.1231e-01,  7.8952e-02,  7.8960e-02, -2.0569e-04,  5.7149e-02,
          5.0050e-03],
        [ 3.0707e-01,  8.2581e-02,  8.2800e-02,  2.3079e-03,  6.3705e-02,
          5.5682e-03],
        [ 1.8268e-01,  1.7962e-02,  1.6763e-02, -1.4421e-02, -9.3755e-03,
         -7.5910e-04],
        [ 1.2628e-01,  1.2482e-01,  1.2879e-01,  4.6993e-02,  1.6402e-01,
          1.4160e-02],
        [ 2.4420e-01, -1.3052e-02, -1.6234e-02, -3.8059e-02, -6.8750e-02,
         -5.8567e-03]], grad_fn=<MmBackward0>)


In [15]:
mask=torch.triu(torch.ones(context_length,context_length),diagonal=1)
print(mask)
masked=attn_score.masked_fill(mask.bool(),-torch.inf)
print(masked)

tensor([[0., 1., 1., 1., 1., 1.],
        [0., 0., 1., 1., 1., 1.],
        [0., 0., 0., 1., 1., 1.],
        [0., 0., 0., 0., 1., 1.],
        [0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0.]])
tensor([[ 0.1791,    -inf,    -inf,    -inf,    -inf,    -inf],
        [ 0.3123,  0.0790,    -inf,    -inf,    -inf,    -inf],
        [ 0.3071,  0.0826,  0.0828,    -inf,    -inf,    -inf],
        [ 0.1827,  0.0180,  0.0168, -0.0144,    -inf,    -inf],
        [ 0.1263,  0.1248,  0.1288,  0.0470,  0.1640,    -inf],
        [ 0.2442, -0.0131, -0.0162, -0.0381, -0.0688, -0.0059]],
       grad_fn=<MaskedFillBackward0>)


In [16]:
print(attn_score*mask1)

tensor([[ 0.1791,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.3123,  0.0790,  0.0000, -0.0000,  0.0000,  0.0000],
        [ 0.3071,  0.0826,  0.0828,  0.0000,  0.0000,  0.0000],
        [ 0.1827,  0.0180,  0.0168, -0.0144, -0.0000, -0.0000],
        [ 0.1263,  0.1248,  0.1288,  0.0470,  0.1640,  0.0000],
        [ 0.2442, -0.0131, -0.0162, -0.0381, -0.0688, -0.0059]],
       grad_fn=<MulBackward0>)


In [17]:
masked_attn_weight=torch.softmax(masked/keys.shape[1],dim=1)
print(masked_attn_weight)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5291, 0.4709, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3587, 0.3206, 0.3207, 0.0000, 0.0000, 0.0000],
        [0.2668, 0.2457, 0.2456, 0.2418, 0.0000, 0.0000],
        [0.2008, 0.2006, 0.2010, 0.1930, 0.2046, 0.0000],
        [0.1865, 0.1640, 0.1637, 0.1619, 0.1594, 0.1645]],
       grad_fn=<SoftmaxBackward0>)


# Masking additional attention weight with Dropout

In [18]:
torch.manual_seed(123)
dropout=torch.nn.Dropout(0.5)
print(dropout)
ones=torch.ones(6,6)
print(ones)
print(dropout(ones))

Dropout(p=0.5, inplace=False)
tensor([[1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.]])
tensor([[2., 2., 0., 2., 2., 0.],
        [0., 0., 0., 2., 0., 2.],
        [2., 2., 2., 2., 0., 2.],
        [0., 2., 2., 0., 0., 2.],
        [0., 2., 0., 2., 0., 2.],
        [0., 2., 2., 2., 2., 0.]])


In [19]:
attn_dropout=dropout(attn_weight)
print(attn_dropout)

tensor([[0.3491, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.3301, 0.0000, 0.3251, 0.3133],
        [0.0000, 0.3305, 0.3306, 0.3123, 0.3262, 0.3130],
        [0.0000, 0.0000, 0.3293, 0.3222, 0.0000, 0.3253],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.3483, 0.3133],
        [0.3903, 0.3254, 0.0000, 0.3197, 0.3128, 0.3271]],
       grad_fn=<MulBackward0>)


# Causal attention Class with Batches

In [42]:
batch=torch.stack((inputs,inputs),dim=0)
print(batch)
print(batch.shape)

tensor([[[0.4300, 0.1500, 0.8900],
         [0.5500, 0.8700, 0.6600],
         [0.5700, 0.8500, 0.6400],
         [0.2200, 0.5800, 0.3300],
         [0.7700, 0.2500, 0.1000],
         [0.0500, 0.8000, 0.5500]],

        [[0.4300, 0.1500, 0.8900],
         [0.5500, 0.8700, 0.6600],
         [0.5700, 0.8500, 0.6400],
         [0.2200, 0.5800, 0.3300],
         [0.7700, 0.2500, 0.1000],
         [0.0500, 0.8000, 0.5500]]])
torch.Size([2, 6, 3])


In [ ]:
print(batch.transpose(1,2))#change row 1 with colmn 2

tensor([[[0.4300, 0.5500, 0.5700, 0.2200, 0.7700, 0.0500],
         [0.1500, 0.8700, 0.8500, 0.5800, 0.2500, 0.8000],
         [0.8900, 0.6600, 0.6400, 0.3300, 0.1000, 0.5500]],

        [[0.4300, 0.5500, 0.5700, 0.2200, 0.7700, 0.0500],
         [0.1500, 0.8700, 0.8500, 0.5800, 0.2500, 0.8000],
         [0.8900, 0.6600, 0.6400, 0.3300, 0.1000, 0.5500]]])


In [67]:
class Causal_Attention(torch.nn.Module):
    def __init__(self,in_d,out_d,context_length,dropout_percent):
        super().__init__()
        self.w_q=torch.nn.Linear(in_d,out_d)
        self.w_k=torch.nn.Linear(in_d,out_d)
        self.w_v=torch.nn.Linear(in_d,out_d)
        self.dropout=torch.nn.Dropout(dropout_percent)
        self.register_buffer('mask',torch.triu(torch.ones(context_length,context_length),diagonal=1))

    
    def forward(self,input_embedding):
        
        b,num_token,d_in=input_embedding.shape
        queries=self.w_q(input_embedding)
        keys=self.w_k(input_embedding)
        values=self.w_v(input_embedding)

        print(keys.shape[-1])

        attn_score=queries@keys.transpose(1,2)

        attn_score.masked_fill_(self.mask.bool()[:num_token, :num_token],-torch.inf)
        
        print('attn_score',attn_score)
        attn_weight=torch.softmax(attn_score/keys[-1]**0.5,dim=-1)
        print('attn_weight',attn_weight)
        attn_weight=self.dropout(attn_weight)

        masked_context_vectore=attn_weight@values
        return masked_context_vectore

        
        

        
        


In [58]:
in_d,out_d=3,2
print(in_d)

3


In [68]:
torch.manual_seed(123)
context_length=batch.shape[1]
print(context_length)
ca=Causal_Attention(in_d,out_d,context_length,0.0)
print(ca)
context_vec=ca.forward(batch)

6
Causal_Attention(
  (w_q): Linear(in_features=3, out_features=2, bias=True)
  (w_k): Linear(in_features=3, out_features=2, bias=True)
  (w_v): Linear(in_features=3, out_features=2, bias=True)
  (dropout): Dropout(p=0.0, inplace=False)
)
2
attn_score tensor([[[0.4271,   -inf,   -inf,   -inf,   -inf,   -inf],
         [0.5730, 0.5534,   -inf,   -inf,   -inf,   -inf],
         [0.5703, 0.5495, 0.5453,   -inf,   -inf,   -inf],
         [0.5143, 0.5440, 0.5368, 0.3764,   -inf,   -inf],
         [0.4756, 0.4463, 0.4436, 0.3051, 0.3194,   -inf],
         [0.5452, 0.5841, 0.5760, 0.4047, 0.3213, 0.4979]],

        [[0.4271,   -inf,   -inf,   -inf,   -inf,   -inf],
         [0.5730, 0.5534,   -inf,   -inf,   -inf,   -inf],
         [0.5703, 0.5495, 0.5453,   -inf,   -inf,   -inf],
         [0.5143, 0.5440, 0.5368, 0.3764,   -inf,   -inf],
         [0.4756, 0.4463, 0.4436, 0.3051, 0.3194,   -inf],
         [0.5452, 0.5841, 0.5760, 0.4047, 0.3213, 0.4979]]],
       grad_fn=<MaskedFillBackward0>

RuntimeError: The size of tensor a (6) must match the size of tensor b (2) at non-singleton dimension 2

In [ ]:
x=[[1,2,3],[4,5,6],[7,8,9]]
y=torch.zeros(3,3)
for r in range(3):
    row=x[r]
    row[r+1:y.append(row)]=row[r+1:]*0
    y[r][r]
    
print(y)

   



tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])
